In [58]:
import helpers
import numpy as np
import math
import re
from copy import deepcopy

data = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""
with open("input_16.txt", 'r') as f:
    data = f.read().rstrip()
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = [changeDict[x.split(" ")[0]] * int(x.split(" ")[1]) for x in data.split("\n") if x]
#data = [x.split('\n') for x in data.split('\n\n') if x]
#np.prod(sum(data))    
state = {}
flow_dictionary = {}
connected_dictionary = {}
state['valves_on'] = {}
for line in data.split('\n'):
    valves = line.split('valve')[1].replace(',', '')
    valves = [x for x in valves.split(' ') if len(x) > 1]
    split_line = line.split(' ')
    valve = split_line[1]
    flow = int(split_line[4].replace(';', '').split('=')[1])
    flow_dictionary[valve] = flow
    connected_dictionary[valve] = valves
    if flow == 0:
        state['valves_on'][valve] = True
    else:
        state['valves_on'][valve] = False
print(state)  
print(flow_dictionary)
connected_dictionary
    

{'valves_on': {'VR': False, 'UV': True, 'OH': True, 'GD': True, 'NS': True, 'KZ': False, 'AH': True, 'SA': True, 'VK': True, 'GB': True, 'AJ': False, 'PJ': True, 'KO': True, 'AA': True, 'TW': True, 'VG': False, 'BP': True, 'XH': True, 'TX': True, 'IR': False, 'TU': True, 'KC': True, 'LN': True, 'HD': True, 'KE': True, 'VY': True, 'LH': True, 'EM': True, 'SO': False, 'EC': True, 'KV': True, 'FE': True, 'TI': True, 'SC': False, 'ZP': True, 'RO': False, 'ZR': True, 'JL': False, 'PW': False, 'UL': True, 'GN': True, 'TN': True, 'NV': True, 'DI': False, 'IC': True, 'JF': True, 'LE': True, 'BS': True, 'JD': False, 'SP': False, 'NY': True, 'OM': False, 'GH': True, 'RI': False}}
{'VR': 11, 'UV': 0, 'OH': 0, 'GD': 0, 'NS': 0, 'KZ': 18, 'AH': 0, 'SA': 0, 'VK': 0, 'GB': 0, 'AJ': 6, 'PJ': 0, 'KO': 0, 'AA': 0, 'TW': 0, 'VG': 25, 'BP': 0, 'XH': 0, 'TX': 0, 'IR': 10, 'TU': 0, 'KC': 0, 'LN': 0, 'HD': 0, 'KE': 0, 'VY': 0, 'LH': 0, 'EM': 0, 'SO': 22, 'EC': 0, 'KV': 0, 'FE': 0, 'TI': 0, 'SC': 14, 'ZP': 0,

{'VR': ['LH', 'KV', 'BP'],
 'UV': ['GH', 'RO'],
 'OH': ['AJ', 'NY'],
 'GD': ['TX', 'PW'],
 'NS': ['AJ', 'AA'],
 'KZ': ['KO', 'VK', 'PJ'],
 'AH': ['ZP', 'DI'],
 'SA': ['VG', 'JF'],
 'VK': ['RO', 'KZ'],
 'GB': ['XH', 'AA'],
 'AJ': ['IC', 'OH', 'ZR', 'NS', 'EM'],
 'PJ': ['KZ', 'SP'],
 'KO': ['KZ', 'LE'],
 'AA': ['TW', 'GB', 'TI', 'NS', 'UL'],
 'TW': ['TU', 'AA'],
 'VG': ['SA'],
 'BP': ['RO', 'VR'],
 'XH': ['GB', 'RI'],
 'TX': ['RI', 'GD'],
 'IR': ['TN', 'NY', 'JF'],
 'TU': ['JD', 'TW'],
 'KC': ['SP', 'RO'],
 'LN': ['EM', 'RI'],
 'HD': ['FE', 'SC'],
 'KE': ['OM', 'RI'],
 'VY': ['PW', 'BS'],
 'LH': ['OM', 'VR'],
 'EM': ['AJ', 'LN'],
 'SO': ['ZP', 'FE'],
 'EC': ['OM', 'UL'],
 'KV': ['SP', 'VR'],
 'FE': ['SO', 'HD'],
 'TI': ['AA', 'PW'],
 'SC': ['HD'],
 'ZP': ['SO', 'AH'],
 'RO': ['UV', 'BP', 'VK', 'KC'],
 'ZR': ['OM', 'AJ'],
 'JL': ['GN', 'TN'],
 'PW': ['TI', 'GN', 'VY', 'GD', 'IC'],
 'UL': ['EC', 'AA'],
 'GN': ['JL', 'PW'],
 'TN': ['JL', 'IR'],
 'NV': ['RI', 'JD'],
 'DI': ['LE', 'AH'],
 'IC

In [59]:
state['time'] = 30
state['pressure'] = 0
max_pressure = 0

def move(state, position, cameFrom):
    #print(state['time'])
    global max_pressure
    state = deepcopy(state)
    if state['time'] == 0 or all(i for i in state['valves_on'].values()) or ((max_score(state, state['time']) + state['pressure']) <= max_pressure):
        if state['pressure'] > max_pressure:
            max_pressure = state['pressure']
            print(max_pressure)
        return
    
    if not state['valves_on'][position]:
        state2 = deepcopy(state)
        state2['valves_on'][position] = True
        state2['time'] -= 1
        state2['pressure'] += state2['time'] * flow_dictionary[position]
        move(state2, position, "")
        
    for movement in connected_dictionary[position]:
        if not movement == cameFrom:
            state2 = deepcopy(state)
            state2['time'] -= 1
            move(state2, movement, position)
    
    
        
move(state, "AA", "")        
        
max_pressure

459
468
487
493
498
522
534
606
616
666
672
696
707
717
750
798
808
833
837
851
860
876
886
890
918
927
936
948
966
980
1061
1086
1097
1191
1217
1253
1286
1298
1304
1399
1481
1533
1577
1621
1622
1658
1732
1760
1766
1796
1852
1880


1880

In [ ]:
1577 too low

In [25]:
all(i for i in state['valves_on'].values())

True

In [43]:
def max_score(state, time):
    scores = []
    for k,v in state['valves_on'].items():
        if not v:
            scores.append(flow_dictionary[k])
    scores.sort()
    scores = reversed(scores)
    score = 0
    for val in scores:
        if time <= 0:
            break
        time -= 1
        score += time * val
        time -= 1
    return(score)

In [55]:
max_score(state, 10)

487